In [1]:
#INSTALACIÓN DE LIBRERIAS
!pip install Torch==1.8.1
!pip install TorchVision

ERROR: Could not find a version that satisfies the requirement Torch==1.8.1 (from versions: 2.0.0, 2.0.1)
ERROR: No matching distribution found for Torch==1.8.1


In [2]:
#IMPORTS
import os
import shutil
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import ParameterGrid

In [4]:
# Test if pickle is correct
# Train data
with open('../IDC_train.dat', 'rb') as f:
    (X_train, y_train) = pickle.load(f)

# Test data
with open('../IDC_test.dat', 'rb') as f:
    (X_test, y_test) = pickle.load(f)

In [ ]:
# Convertir datos de entrenamiento a tensores
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

# Convertir datos de prueba a tensores
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [23]:
# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define the model architecture
class Model(nn.Module):
    def __init__(self, num_neurons):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, num_neurons, 3)
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        
        self.fc = nn.Linear(num_neurons*50*50*3, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.flatten(x)
            
        x = self.fc(x)
        x = self.sigmoid(x)
        return x


# Define the function to create the model
# Define the function to create the model
def crear_modelo(num_neurons):
    model = Model(num_neurons)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    return model, criterion, optimizer

# Define the grid of parameters
param_grid = {
    'num_neurons': [10, 20, 30, 40, 50],
}
# Perform grid search
best_accuracy = 0.0
best_params = None

In [24]:
X_train.shape

torch.Size([222019, 50, 50, 3])

In [25]:
# Forward Pass para probar si la red funciona
model = Model(10)

batch_size = 1
input_shape = (3, 224, 224)
inputs = torch.randn(batch_size, *input_shape)

outputs = model(inputs)

print(outputs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x123210 and 75000x1)

In [27]:
inputs

tensor([[[[ 0.7173, -0.7130, -0.4417,  ...,  2.0409,  1.9807,  0.6817],
          [ 1.8647,  0.2237, -0.3349,  ..., -1.1041, -1.9769, -0.8542],
          [ 0.8280, -0.8898,  0.1478,  ..., -1.0272,  0.1604, -0.4046],
          ...,
          [-1.9202, -0.2081,  1.1455,  ..., -1.1719, -0.4182, -2.4341],
          [ 0.6424,  0.8525, -0.4016,  ..., -0.2898, -0.3073,  0.3027],
          [ 0.1467,  0.6620,  0.6757,  ...,  0.8763, -0.0287, -0.7981]],

         [[ 0.5437,  0.7964, -0.7785,  ..., -1.0187, -1.0723,  0.9971],
          [-0.6546,  0.3824, -0.1794,  ..., -0.9837,  0.0746, -0.0525],
          [-1.0634, -0.6565,  0.2763,  ..., -1.5390,  0.3812, -0.5767],
          ...,
          [-0.7119, -0.3076,  0.3942,  ...,  0.4839, -0.6875, -0.1747],
          [ 1.2536, -1.2595,  1.2237,  ...,  0.7604, -0.5051,  0.9017],
          [ 0.1850, -0.4167,  0.7353,  ...,  2.1681,  0.7902, -0.4878]],

         [[-0.6784, -1.6887,  0.0856,  ..., -0.4539,  0.3251,  1.1501],
          [ 1.4816,  0.0906, -

In [ ]:
for params in ParameterGrid(param_grid):
    model, criterion, optimizer = crear_modelo(params['num_neurons'], params['hidden_size'], params['num_layers'])

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    for epoch in range(10):
        model.train()
        image_counter = 0
        for inputs, labels in train_loader:
            image_counter += inputs.size(0)
            print(f"Epoch: {epoch+1}, Processed images: {image_counter}")
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                predicted = (outputs >= 0.5).squeeze().long()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)